In [ ]:
if (!require("BiocManager", quietly = TRUE)) install.packages("BiocManager")
  BiocManager::install("ggrepel")
  BiocManager::install("tidyverse")

In [ ]:
library(ggplot2)
library(ggrepel)  #用于标记的包
library(tidyverse)

In [ ]:
# 读取火山图数据文件
data <- read.delim("https://www.bioladder.cn/shiny/zyp/bioladder2/demoData/Volcano/volcano2.txt")

# 建议您的文件里对应的名称跟demo数据一致，这样不用更改后续代码中的变量名称

FC <- 1.2 # 用来判断上下调，一般蛋白质组的项目卡1.5
PValue <- 0.05 # 用来判断上下调

# 标记方式（一）
# 根据数据框中的Marker列，1的为标记，其他的为不标记
data$label <- ifelse(data$Marker == 1, as.character(data$Name), "")
# （或）标记方式（二）
# 根据PValue小于多少和log[2]FC的绝对值大于多少筛选出合适的点
# PvalueLimit = 0.0001
# FCLimit = 5
# data$label=ifelse(data$PValue < PvalueLimit & abs(log2(data$FC)) >= FCLimit, as.character(data$Name), '')

# 绘图
ggplot(data, aes(log2(FC), -1 * log10(PValue))) + # 加载数据，定义横纵坐标
    geom_point(aes(color = -1 * log10(PValue), size = -1 * log10(PValue))) + # 绘制散点图，颜色和大小依据是数据框的PValue列
    scale_color_gradientn(colors = c(
        "#5E4FA2", "#3288BD", "#66C2A5", # 设置渐变色
        "#ABDDA4", "#E6F598", "#FFFFBF",
        "#FEE08B", "#FDAE61", "#F46D43",
        "#D53E4F", "red", "#9E0142"
    )) +
    scale_radius( # 设置点大小图例
        range = c(1, 3), # 设置点大小的范围
        guide = NULL
    ) + # 不显示该图例
    labs(
        title = "", # 定义标题，x轴，y轴名称
        x = "log[2](FC)",
        y = "-log[10](PValue)"
    ) +
    geom_hline(yintercept = -log10(PValue), linetype = 2) + # 在图上添加虚线
    geom_vline(xintercept = c(-log2(FC), log2(FC)), linetype = 2) + # 在图上添加虚线
    geom_text_repel(
        aes(
            x = log2(FC), # geom_text_repel 标记函数
            y = -1 * log10(PValue),
            label = label
        ),
        max.overlaps = 10000, # 最大覆盖率，当点很多时，有些标记会被覆盖，调大该值则不被覆盖，反之。
        size = 3, # 字体大小
        box.padding = unit(0.5, "lines"), # 标记的边距
        point.padding = unit(0.1, "lines"),
        segment.color = "black", # 标记线条的颜色
        show.legend = FALSE
    ) +
    theme_classic()
